In [14]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [15]:
import pymysql
pymysql.install_as_MySQLdb()

In [16]:
csv_file= "Dockless_Vehicle_Trips.csv"
dockless_data_df = pd.read_csv(csv_file)
dockless_data_df.head()

,ID,Device ID,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,Modified Date,Month,Hour,Day of Week,Council District (Start),Council District (End),Origin Cell ID,Destination Cell ID,Year,Start Latitude,Start Longitude,End Latitude,End Longitude
0,000000cb-78a9-4b95-8ead-37c797abcf37,ad14e63b-940d-4ca3-84c6-0131147fba45,scooter,358.0,915.0,6/29/2018 17:30,6/29/2018 17:30,12/12/2018 21:36,6.0,17.0,5.0,5,9,13445,13916,2018.0,30.263,-97.762,30.259,-97.754
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,c5a230c9-7e80-4464-862f-f2b23f61d6d3,scooter,226.0,839.0,11/30/2018 11:00,11/30/2018 11:00,12/13/2018 15:43,11.0,11.0,5.0,9,9,14556,15186,2018.0,30.283,-97.745,30.284,-97.737
2,00002334-2f2b-41a0-9a04-8ff66be39f40,5a9f7815-bf0c-4fef-8bac-60efd4f23b10,scooter,324.0,1206.0,10/2/2018 15:15,10/2/2018 15:30,12/13/2018 15:49,10.0,15.0,2.0,9,9,15344,15032,2018.0,30.285,-97.734,30.291,-97.738
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,41fa0cf0-f433-4b3a-89f5-2c6cec6c367d,scooter,1096.0,0.0,9/2/2018 16:45,9/2/2018 17:00,12/13/2018 15:53,9.0,16.0,0.0,9,9,14707,14081,2018.0,30.268,-97.743,30.276,-97.752
4,00002c70-678b-4c09-a32c-da0266e329c6,ddd4bbc3-114e-4cbc-b078-860e3c28b11b,scooter,408.0,1144.0,9/28/2018 11:45,9/28/2018 11:45,12/12/2018 22:24,9.0,11.0,5.0,3,3,16114,15642,2018.0,30.242,-97.721,30.244,-97.729


In [30]:
#from datetime import datetime
#ts = int("1284101485")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
#print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))
#i = []
#for i in len(dockless_data_df)

#date1_test = dockless_data_df['Start Time'][i].split(' ')[0]
# time1_test = dockless_data_df['Start Time'][0].split(' ')[1]

# date1_test, time1_test

2010-09-10 06:51:25


In [17]:
new_dockless_data_df = dockless_data_df.rename(columns={'Trip Duration':'Trip_in_seconds', 
                                                        'Trip Distance':'Trip_in_meters',
                                                        'Start Time':'trip_start',
                                                        'End Time': 'Trip_ended'})
#new_dockless_data_df.drop_duplicates('id', inplace=True)
#new_license_data_df.set_index('id', inplace=True)
#new_license_data_df = license_premise_df[['id', 'first_name', 'last_name']].copy()

new_dockless_data_df.head()

,ID,Device ID,Vehicle Type,Trip_in_seconds,Trip_in_meters,trip_start,Trip_ended,Modified Date,Month,Hour,Day of Week,Council District (Start),Council District (End),Origin Cell ID,Destination Cell ID,Year,Start Latitude,Start Longitude,End Latitude,End Longitude
0,000000cb-78a9-4b95-8ead-37c797abcf37,ad14e63b-940d-4ca3-84c6-0131147fba45,scooter,358.0,915.0,6/29/2018 17:30,6/29/2018 17:30,12/12/2018 21:36,6.0,17.0,5.0,5,9,13445,13916,2018.0,30.263,-97.762,30.259,-97.754
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,c5a230c9-7e80-4464-862f-f2b23f61d6d3,scooter,226.0,839.0,11/30/2018 11:00,11/30/2018 11:00,12/13/2018 15:43,11.0,11.0,5.0,9,9,14556,15186,2018.0,30.283,-97.745,30.284,-97.737
2,00002334-2f2b-41a0-9a04-8ff66be39f40,5a9f7815-bf0c-4fef-8bac-60efd4f23b10,scooter,324.0,1206.0,10/2/2018 15:15,10/2/2018 15:30,12/13/2018 15:49,10.0,15.0,2.0,9,9,15344,15032,2018.0,30.285,-97.734,30.291,-97.738
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,41fa0cf0-f433-4b3a-89f5-2c6cec6c367d,scooter,1096.0,0.0,9/2/2018 16:45,9/2/2018 17:00,12/13/2018 15:53,9.0,16.0,0.0,9,9,14707,14081,2018.0,30.268,-97.743,30.276,-97.752
4,00002c70-678b-4c09-a32c-da0266e329c6,ddd4bbc3-114e-4cbc-b078-860e3c28b11b,scooter,408.0,1144.0,9/28/2018 11:45,9/28/2018 11:45,12/12/2018 22:24,9.0,11.0,5.0,3,3,16114,15642,2018.0,30.242,-97.721,30.244,-97.729


In [19]:
new_dockless_df = new_dockless_data_df[["ID", "Vehicle Type", "Trip_in_seconds", "Trip_in_meters", "trip_start", "Trip_ended"]].copy()
new_dockless_df.head()





,ID,Vehicle Type,Trip_in_seconds,Trip_in_meters,trip_start,Trip_ended
0,000000cb-78a9-4b95-8ead-37c797abcf37,scooter,358.0,915.0,6/29/2018 17:30,6/29/2018 17:30
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,scooter,226.0,839.0,11/30/2018 11:00,11/30/2018 11:00
2,00002334-2f2b-41a0-9a04-8ff66be39f40,scooter,324.0,1206.0,10/2/2018 15:15,10/2/2018 15:30
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,scooter,1096.0,0.0,9/2/2018 16:45,9/2/2018 17:00
4,00002c70-678b-4c09-a32c-da0266e329c6,scooter,408.0,1144.0,9/28/2018 11:45,9/28/2018 11:45


In [ ]:
new_dockless_df['Date'] = pd.to_datetime(df['trip_start'])
new_dockless_df = pd.DataFrame({'Date':pd.date_range('2018-01-01', periods = 60, freq='D')})

new_dockless_df.Date.dt.strftime('%Y%M%D').astype(int)